In [15]:
import nltk
import numpy as np
import urllib.request
import random
import string
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords  

In [16]:
#obtener el contenido HTML de la pagina web 
res = urllib.request.urlopen("https://www.ionos.es/digitalguide/paginas-web/desarrollo-web/que-es-el-web-scraping/")
html = res.read()
soup = BeautifulSoup(html,"html.parser")

In [17]:
#observamos la etiquetas HTML
setTag = {x.name for x in soup.find_all(True)}
print(setTag)

{'source', 'style', 'article', 'nav', 'body', 'button', 'i', 'a', 'br', 'html', 'link', 'form', 'input', 'h1', 'h3', 'time', 'label', 'header', 'ol', 'img', 'noscript', 'h2', 'ul', 'picture', 'h4', 'main', 'li', 'head', 'div', 'base', 'p', 'meta', 'section', 'footer', 'span', 'title', 'fieldset', 'em', 'script', 'strong'}


In [18]:
Content = ''
for tag in soup.find_all('p'):
    if(len(tag.text) > 144):
        Content += tag.text+'\n\n'
print(Content)

Los motores de búsqueda, como Google, utilizan desde hace tiempo los denominados rastreadores web o crawlers, que exploran Internet en busca de términos definidos por el usuario. Los rastreadores son tipos especiales de bots, que visitan una página web tras otra para generar asociaciones con términos de búsqueda y categorizarlos. El primer rastreador web se creó ya en 1993, cuando se presentó el primer motor de búsqueda: Jumpstation.

Entre estas técnicas de rastreo se incluye el web scraping o webharvesting. Te explicamos cómo funciona, para qué se utiliza y cómo se puede bloquear en caso necesario.

Durante el web scraping (del inglés scraping = arañar/raspar) se extraen y almacenan datos de páginas web para analizarlos o utilizarlos en otra parte. Por medio de este raspado web se almacenan diversos tipos de información: por ejemplo, datos de contacto, tales como direcciones de correo electrónico o números de teléfono, o también términos de búsqueda o URL. Estos se almacenan en bases

In [19]:
raw = Content.lower()

In [20]:
sent_tokens = nltk.sent_tokenize(raw) # convertir corpus a una lista de sentencias (tekenizar en sentencias)
word_tokens = nltk.word_tokenize(raw) # convertir corpus a una lista de palabras (tokenizar en palabras)

lemmer = nltk.stem.WordNetLemmatizer()#

In [21]:
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation) #remover signos de puntuacion (utilizamos la )

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens] # Lematizar palabras tokenizadas

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [22]:
#Determinar la similutud del texto inserdado y el corpus

def response(user_response):# funcion respuesta (entrada: mensje de usuario)
    chatbot_response = '' #definir la respuesta del chatbot
    sent_tokens.append(user_response)# al listado de sentencias del corpus añadir al final de la lista el mensaje del usuario
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize,stop_words=stopwords.words('spanish'))# Lematizar y eliminar palbras
    tfidf = TfidfVec.fit_transform(sent_tokens)

    # print("a",tfidf[-1])
    # print("------------------")
    # print(tfidf)
    # 3 Evaluar similitud coseno entre mensaje usuario (tfid[-1]) y el corpus(tfidf)
    vals = cosine_similarity(tfidf[-1],tfidf)
   

    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    
    flat.sort()
  
    req_tfidf = flat[-2]
    # print("------------------")
    # print(req_tfidf)
    # print(idx)
    if(req_tfidf == 0):
        chatbot_response = chatbot_response + "Lo siento, no te he entendido"
        return chatbot_response
    
    else:
        chatbot_response = chatbot_response + sent_tokens[idx]
        return chatbot_response

In [23]:
SALUDOS_INPUTS = ("hola","buenas","saludos","que tal","hey","buenos dias")
SALUDOS_OUTPUTS = ["hola","hola, que tal?","Hola, Como te puedo ayudar?","hola, encantado de hablar contigo"]

def saludos(sentence):
    for word in sentence.split():
        # print(word.lower() in SALUDOS_INPUTS)
        if word.lower() in SALUDOS_INPUTS:
            return random.choice(SALUDOS_OUTPUTS)

In [24]:
flag = True

print("AGENTE: MI NOMBRE ES AGENTE. CONTESTARE A SUS PREGUNTAS ACERCA DEL COVID Y LAS MEDIDAS DE SIGUIRDAD.")
while(flag == True):
    user_response = input()
    user_response = user_response.lower() #convertimos a minuscula

    if(user_response not in ["salir","chao","adios","bay","exit"]):
        if(user_response == 'gracias' or user_response == 'muchas gracias'): #se podria haber definido otra funcion de coincidencia
            flag = True
            print("AGENTE: No hay de que")
        else:
            if(saludos(user_response) != None): # si la palabra insertada por el usuario es un saludo (coincidencia manuales)
                print("AGENTE: "+saludos(user_response))
            else:#si la palabra insertada no es un saludo --> Corpus
                print("AGENTE: ",end ="*")
                print(response(user_response),end="*\n")
                sent_tokens.remove(user_response) # para eliminar del corpus la respues del usuario y volver a evaluar 
    else:
        flag = False
        print("AGENTE: Nos vemos pronto, !Gracias por su visita¡")    

AGENTE: MI NOMBRE ES AGENTE. CONTESTARE A SUS PREGUNTAS ACERCA DEL COVID Y LAS MEDIDAS DE SIGUIRDAD.
AGENTE: *para garantizar la legalidad del web scraping, hay que tener en consideración lo siguiente:

aunque el scraping está permitido en muchos casos, puede utilizarse con fines destructivos o incluso ilegales.*
AGENTE: *el scrapingno siempre es legal.*
AGENTE: *Lo siento, no te he entendido*
AGENTE: Nos vemos pronto, !Gracias por su visita¡
